# Letter Counter

oh yeah!



### Loading Assets



In [1]:
import unidecode
from collections import defaultdict


### Configuration

If you want to change the source of the data, do it here.

The format is csv, with one column, with heading (1st row is ignored)


In [2]:
filename = "../data/ListaTodasPalabras.csv"



### Counting characters



In [3]:
# Initialise some variables
d = defaultdict(lambda:0)
total = 0

with open(filename) as fh:
    # count the total words and the letters
    for word in fh.readlines():
        if word == "PALABRAS\n":
            continue # Ignore the header name
        total += 1
        noacc = unidecode.unidecode(word)
        for letter in noacc.strip():
            d[letter] += 1



In [4]:
print("Letter frequency from least to most")
s = {k: v for k, v in sorted(d.items(), key=lambda item: item[1])}


for item,val in s.items():
    print(f"{item} : {val}")

print("Total words counted")
print(total)

Letter frequency from least to most
w : 24
k : 89
y : 1252
x : 1402
q : 3280
j : 5190
z : 6454
v : 8209
h : 8224
f : 9330
g : 14703
b : 15241
p : 19532
m : 25658
u : 26367
d : 34490
s : 34923
l : 39449
t : 43949
c : 46648
n : 52935
i : 70377
r : 77056
o : 78651
e : 79179
a : 130353
Total words counted
95383


# Empezando de cero

In [5]:
import pandas as pd
import matplotlib


df = pd.read_csv(filename)

df

,PALABRAS
0,aba
1,aca
2,acá
3,adó
4,afé
...,...
95378,ultramicroscopio
95379,ventriculografía
95380,vertiginosamente
95381,vicealmirantazgo


In [6]:

splits = df.apply(lambda x: [y for y in x[0].strip()], axis='columns')
splits

0                                               [a, b, a]
1                                               [a, c, a]
2                                               [a, c, á]
3                                               [a, d, ó]
4                                               [a, f, é]
                               ...                       
95378    [u, l, t, r, a, m, i, c, r, o, s, c, o, p, i, o]
95379    [v, e, n, t, r, i, c, u, l, o, g, r, a, f, í, a]
95380    [v, e, r, t, i, g, i, n, o, s, a, m, e, n, t, e]
95381    [v, i, c, e, a, l, m, i, r, a, n, t, a, z, g, o]
95382    [v, i, t, u, p, e, r, i, o, s, a, m, e, n, t, e]
Length: 95383, dtype: object

In [15]:
d = defaultdict(lambda:0)

def ccount(word=None):
    assert word is not None
    for letter in word:
        d[letter] += 1
    return word

splits.apply(ccount)


0                                               [a, b, a]
1                                               [a, c, a]
2                                               [a, c, á]
3                                               [a, d, ó]
4                                               [a, f, é]
                               ...                       
95378    [u, l, t, r, a, m, i, c, r, o, s, c, o, p, i, o]
95379    [v, e, n, t, r, i, c, u, l, o, g, r, a, f, í, a]
95380    [v, e, r, t, i, g, i, n, o, s, a, m, e, n, t, e]
95381    [v, i, c, e, a, l, m, i, r, a, n, t, a, z, g, o]
95382    [v, i, t, u, p, e, r, i, o, s, a, m, e, n, t, e]
Length: 95383, dtype: object

In [16]:
d

defaultdict(<function __main__.<lambda>()>,
            {'a': 127140,
             'b': 15241,
             'c': 46648,
             'á': 3213,
             'd': 34490,
             'ó': 6768,
             'f': 9330,
             'é': 2098,
             'g': 14703,
             'h': 8224,
             'í': 5363,
             'j': 5190,
             'e': 77081,
             'o': 71883,
             'l': 39449,
             'i': 65014,
             'm': 25658,
             'n': 50682,
             'ñ': 2253,
             'p': 19532,
             'r': 77056,
             's': 34923,
             't': 43949,
             'u': 25552,
             'v': 8209,
             'x': 1402,
             'y': 1252,
             'ú': 621,
             'z': 6454,
             'k': 89,
             'q': 3280,
             'w': 24,
             'ü': 194})